<a href="https://colab.research.google.com/github/cjfghk5697/multi-label-classification-Mnist/blob/main/train%20Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 이진화 코드 추가
 이진화란 이미지를 흰색과 검은색으로 바꾼다고 생각하면 된다.
속도가 향상되고 정확도도 미약하지만 향상된것을 볼수 있다.

 [Dacon 적용 예시](https://dacon.io/competitions/official/235697/codeshare/2436?page=2&dtype=recent)


 [이진화 설명](https://gmnam.tistory.com/263)

 <hr>

 # 2. train, valid set 나누기
```
train_dataset, valid_dataset = torch.utils.data.random_split(trainset, [train_size, valid_size])

train_loader = DataLoader(train_dataset, batch_size=64, num_workers=8)
valid_loader = DataLoader(valid_dataset, batch_size=64, num_workers=8)
```
처음에는 나누어지지 않아있는 코드였단 걸 발견하고 random_split 함수를 이용해 나눔. 그리고 학습 부분에서 valid 데이터를 enumerate하는 구문추가해서 valid 확인.

<hr>

# 3. Multi Sample Dropout 구현

[자료](https://velog.io/@sujeongim/%EC%83%88%EB%A1%9C-%EB%B0%B0%EC%9A%B4-%EC%BD%94%EB%93%9C-%EC%A1%B0%EA%B0%81-Multi-Sample-Dropout)
단순 dropout 기법을 안썼다. 근데 큰효과가 있는지는 잘 모르지만 overfitting의 위협도 없앨겸 선택했다.

<hr>

# 4. Scheduler 추가
일단 간단하게 StepLR scheduler을 추가했다.



<hr>

# 5. Drop out 실수
전 Drop out에서는 sigmoid가 있다. 근데 필자가 Multi-Label Loss Function을 이용함으로써 활성화 함수를 더 추가할 필요가 없었다. 그러나 sigmoid를 또 넣음으로써 sigmoid 계층 2개를 들어가는 거다. 결국 loss 값이 적어져서 학습 진행이 안된거다. 학습이 잘 안되면 활성화 함수에서 loss 값이 얼마 나오는지 잘 확인해야겠다..

https://cvml.tistory.com/26 
Multi-Label Loss Function에 대해 잘 설명 되어있다.


<hr>

# 6. 왜 정확도가 안오를까?
이상하게도 valid도 장확도가 같이 오르는데 test를 하면 정확도는 50%다 혹시나 해서 이진화를 빼고 해봤는데도 정확도가 안오른다 (05.15)

-테스트

1. efficientnetb1에서 epoch 수를 늘려봤다. 과거에는 5번 정도만 해도 valid 정확도가 높게 나왔다. 하지만 epoch를 늘린다고 해결될 일이 절대 아닌 거 같다.

2. overfitting 인거 같아서 dropout을 추가했다. 활성화 함수도 한층 추가해봐서 실험. (실패)

3. 추적그래프를 그리기 위해 추적그래프를 추가했다.

4. DACON 토론에 올렸다.[링크](https://dacon.io/forum/406352)  답변이 왔고 확실히 여기서 model.eval()과 model.train(), model.zero_grab()에 대해 다시 공부했다.. valid에 zero_grab을 하여 학습이 진행되고 있었고 eval 또한 안해서 불필요한 과정을 가지고 있었다. 

### 해결
결국은 inference에서 모델을 잘못 가지고 오고 있었다.. 결국 다시해본결과 80% 정확도가 나왔지만 아직 90% 정확도는 안나온다.

# 7. 또다시 연구
이번에는 적절한 파라미터를 찾기위한 튜닝 작업을 해보고자 한다.

일단 Train 코드에서는 augmentation(mix up
)과 여러 scheduler를 추가하려고한다. 이제 휴가가 끝나서 시간 날때마다 추가해야겠다.







In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Train image2
!unzip -q "/content/drive/MyDrive/multi-mnist zipfile/dirty_mnist_2nd.zip"

/content/drive/MyDrive/Train image2


In [ ]:
!pip3 install torchinfo
!pip3 install cv2

In [ ]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
from torchvision import transforms
from torchvision.models import efficientnet_b0
from torch.nn import functional as F




In [ ]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(
                self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)


        #image = np.array(image)
#이진화 완료 cv2 툴을 써서 threshold로 254 값으로 변환. cv2는 넘파이만 사용 가능해서 넘파이로 변환후 fromarry로 pil로 바꾼다.
        #image_th = cv2.threshold(image,254,255,0)[1]
        #image_th = Image.fromarray(image_th) # NumPy array to PIL image
        if self.transforms is not None:
            image = self.transforms(image)

        return image, target

In [ ]:
transforms_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

In [ ]:
trainset = MnistDataset('/content/drive/MyDrive/Train image2', '/content/drive/MyDrive/multi-mnist zipfile/dirty_mnist_2nd_answer.csv', transforms_train)

lengths = [int(len(trainset)*0.8), int(len(trainset)*0.2)]

train_dataset, valid_dataset = torch.utils.data.random_split(trainset, lengths)

train_loader = DataLoader(train_dataset, batch_size=16, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=8, num_workers=2)

In [ ]:
class MnistModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.model = efficientnet_b0(pretrained=True)
        self.dropout=nn.Dropout(p=0.5)
        self.classifier = nn.Linear(1000, 26)
    def forward(self, x):
        #x = F.relu(self.model(x))
        x = self.model(x)  
        x = self.classifier(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MnistModel().to(device)
print(summary(model, input_size=(1, 3, 256, 256), verbose=0))

In [ ]:
class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                e_w = p.grad * scale.to(p)
                p.add_(e_w)  # climb to the local maximum "w + e(w)"
                self.state[p]["e_w"] = e_w

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.sub_(self.state[p]["e_w"])  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def step(self, closure=None):
        assert closure is not None, "Sharpness Aware Minimization requires closure, but it was not provided"
        closure = torch.enable_grad()(closure)  # the closure should do a full forward-backward pass

        self.first_step(zero_grad=True)
        closure()
        self.second_step()

    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        norm = torch.norm(
                    torch.stack([
                        p.grad.norm(p=2).to(shared_device)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm

In [ ]:

criterion = torch.nn.BCELoss()
    #scheduler 추가 및 step 추가
path = "/content/drive/MyDrive/models/"
MODEL = "06052218 eff4"
base_optimizer = torch.optim.SGD 
optimizer = SAM(model.parameters(), base_optimizer,lr=0.001, momentum=0.9)

num_epochs = 33
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=0.001, last_epoch=-1)

valid_loss_list=[]
train_loss_list=[]
for epoch in range(num_epochs):
    train_acc_list = []
    valid_acc_list =[]
    model.train()
    for i, (images, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)
        with torch.set_grad_enabled(True):
          outputs = model(images)
          loss = criterion(outputs, targets)

          loss.backward()
          optimizer.first_step(zero_grad=True)
          criterion(model(images), targets).backward()
          optimizer.second_step(zero_grad=True)
          if (i+1) % 10 == 0:
              outputs = outputs > 0.5
              acc = (outputs == targets).cpu().float().mean()
              train_acc_list.append(acc)
              train_loss_list.append(loss)
              print(f'{epoch}: "train loss:"{loss.item():.5f}, "train acc: "{acc.item():.5f}')
    model.eval()
    with torch.no_grad():
      for i, (images, targets) in enumerate(valid_loader):

          images = images.to(device)
          targets = targets.to(device)

          outputs = model(images)
          valid_loss = criterion(outputs, targets)
      
          if (i+1) % 10 == 0:
              outputs = outputs > 0.5
              valid_acc = (outputs == targets).cpu().float().mean()
              valid_acc_list.append(valid_acc)
              valid_loss_list.append(valid_loss)
              print(f'{epoch}: "val loss:"{valid_loss.item():.5f}, "val acc: "{valid_acc.item():.5f}')
    lr_scheduler.step()
    torch.save(model.state_dict(), f'{path}_{MODEL}_1.pth')

In [ ]:
import matplotlib.pyplot as plt


plt.plot(train_acc_list,color='#e35f62', label='Train Acc')

plt.xlabel('Acc')
plt.ylabel('Epoch')
plt.title('Train')
plt.show()

In [ ]:
plt.plot(valid_acc_list,color='green', label='Valid Acc')
plt.xlabel('Acc')
plt.ylabel('Epoch')
plt.title('Valid')
plt.show()

# 기록
1번 
lr=1e-3

1.   optimizer : Adam
2.   scheduler : CosineAnnealingLR
3.   criterition:MultiLabelSoftMarginLoss


```
9: "val loss:"0.57146, "val acc: "0.67067
9: "val loss:"0.56269, "val acc: "0.67788
9: "val loss:"0.56572, "val acc: "0.67308
9: "val loss:"0.57164, "val acc: "0.66827
9: "val loss:"0.54050, "val acc: "0.73077
9: "val loss:"0.56166, "val acc: "0.66106
9: "val loss:"0.53295, "val acc: "0.68990
9: "val loss:"0.55628, "val acc: "0.68990
9: "val loss:"0.52732, "val acc: "0.71394
```

2번
1번과 같지만 모델이 efficientnet_b5 훨씬 효능이 좋다. 

```
8: "val loss:"0.35732, "val acc: "0.83654
8: "val loss:"0.40119, "val acc: "0.81250
8: "val loss:"0.44620, "val acc: "0.77885
8: "val loss:"0.38490, "val acc: "0.83173
8: "val loss:"0.45323, "val acc: "0.81250
8: "val loss:"0.48173, "val acc: "0.81731
8: "val loss:"0.41693, "val acc: "0.81250
8: "val loss:"0.36138, "val acc: "0.85577
8: "val loss:"0.32753, "val acc: "0.85096
8: "val loss:"0.39352, "val acc: "0.84615
8: "val loss:"0.41519, "val acc: "0.81731
8: "val loss:"0.39349, "val acc: "0.80769
8: "val loss:"0.40276, "val acc: "0.82692
8: "val loss:"0.40279, "val acc: "0.80288
8: "val loss:"0.38277, "val acc: "0.82692
8: "val loss:"0.35343, "val acc: "0.85577
```
